# GleanerIO SHACL multisite processor

## About

Ted Habberman presented to the CDF on ISO metadata analysis.  I have been exploring some approaches to accessing the FAIR Digital Object server that is part of the standard setup supporting GeoCODES and other groups.  Leveraging Python Dask and S3 Boto it is easy to access these object stores in a highly performant manner.  

In addition to DASK there is S3SELECT as well as SPARQL calls on objects and triplestores.  

## Notes

Curious above local context file reading when doing lots of calls.  PyLD seems to know to do caching but not sure how it could in a Dask based pattern.

May need to explore approach to read local files for context like

## Imports


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning

import boto3
import pandas as pd
import json
import s3fs
import kglab
import seaborn as sns
from rdflib import Graph  #, plugin
import getpass
import plotly.express as px
from tqdm import tnrange
from matplotlib import pyplot as plt
import dask
from dask.distributed import Client
import dask.dataframe as dd
import dask.multiprocessing

## Set up Dask client

In [3]:
from dask.distributed import Client    #, progress
client = Client(threads_per_worker=5, n_workers=4)
client

/home/fils/.conda/envs/kglab/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45661 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:45661/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:45661/status,Workers: 4
Total threads: 20,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:46601,Workers: 4
Dashboard: http://127.0.0.1:45661/status,Total threads: 20
Started: Just now,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:42593,Total threads: 5
Dashboard: http://127.0.0.1:45497/status,Memory: 7.79 GiB
Nanny: tcp://127.0.0.1:34837,


## File Access

In [9]:
# Set the shapegraph to use
sg = './shapes/geocodes_dcscan.ttl'

## Anonymous S3 File system
oss = s3fs.S3FileSystem(
    anon=True,
    client_kwargs = {"endpoint_url":"https://oss.geodex.org"}
)

## Access controlled s3
# session = boto3.Session(profile_name='default' ,   region_name="us-east-1")
# s3 = session.client('s3')  # needed later for listing objects
# s3r = session.resource('s3')
# oss = s3fs.S3FileSystem( profile="default")

## Manual code access
# ACCESS_CODE = getpass.getpass()
# SECRET_CODE = getpass.getpass()

# oss = s3fs.S3FileSystem(
#     anon=False,
#     key=ACCESS_CODE,
#     secret=SECRET_CODE,
#     client_kwargs = {"endpoint_url":"http://192.168.86.45:49159"}
# )

In [10]:
# [optional] List the directories we can work with later
sumlist = oss.ls('gleaner/summoned') # gleaner.oih/summoned/
print(sumlist)
print(len(sumlist))

# for testing, set to a smaller subset

# sumlist = ['gleaner/summoned/lipdverse', 'gleaner/summoned/magic', 'gleaner/summoned/hydroshare']

['gleaner/summoned/bcodmo', 'gleaner/summoned/cchdo', 'gleaner/summoned/earthchem', 'gleaner/summoned/getiedadataorg', 'gleaner/summoned/hydroshare', 'gleaner/summoned/ieda', 'gleaner/summoned/iris', 'gleaner/summoned/lipdverse', 'gleaner/summoned/magic', 'gleaner/summoned/ocd', 'gleaner/summoned/opentopo', 'gleaner/summoned/ssdb', 'gleaner/summoned/ucar', 'gleaner/summoned/unavco', 'gleaner/summoned/wikilinkedearth', 'gleaner/summoned/wwwbco-dmoorg', 'gleaner/summoned/wwwhydroshareorg']
17


### Definitions

In [11]:
# some color coding for pandas for later
def change_color_group(x):
    dfcg = x.copy()
    dfcg.loc[df['severity'] == "shacl:Violation", :] = 'background-color: #F89782'
    dfcg.loc[df['severity'] == "shacl:Warning", :] = 'background-color: #F0F480'
    dfcg.loc[df['severity'] == "shacl:Info", :] = 'background-color: #CBFBD2'
    return dfcg

In [12]:
## need a SPARQL for counts of the various validations


bc = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
SELECT  ?tid ?b
  WHERE {
    ?tid rdf:type sh:ValidationReport .
    ?tid sh:conforms ?b .
  }

"""

ccount = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
SELECT  ?b (COUNT(?tid) as ?tcount)
  WHERE {
    ?tid rdf:type sh:ValidationReport .
    ?tid sh:conforms ?b .
  }

group by (?b)
"""

scount = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
SELECT  ?severity (COUNT(?id) as ?tcount)
  WHERE {
    ?id rdf:type sh:ValidationResult .
    ?id sh:resultSeverity ?severity .
  }
group by (?severity)

"""





In [13]:
# SHACL function
@dask.delayed()
def read_and_validate(fn):
    # make a kg to hold our data graph
    namespaces = {
        "schema":  "https://schema.org/",
        "shacl":   "http://www.w3.org/ns/shacl#" ,
    }

    kg = kglab.KnowledgeGraph(
        name = "Schema.org based datagraph",
        base_uri = "https://example.org/id/",
        namespaces = namespaces,
    )

    ## this is a mess.   no need to load to a graph objet to validate.  also, 
    ## can we load up the graph first and then validate later for everythng?
    with oss.open(fn, 'rb') as f:
        jld = json.loads(f.read().decode("utf-8", "ignore").replace('\n',' '))
        jlds = json.dumps(jld)

        g = Graph().parse(data=jlds, format='json-ld')
        context = {
            "@vocab": "https://schema.org/",
        }

        ttl = g.serialize(format='ttl', context=context, indent=4)
        kg.load_rdf_text(ttl, format="ttl", base=None)

        conforms, report_graph, report_text = kg.validate(
            shacl_graph=sg,
            shacl_graph_format="ttl"
        )

        return report_graph.save_rdf_text()  # return graph object or string of graph

## Get files


### Multi Source Loop

In [14]:
# supress warning, use with caution
import warnings 
warnings.filterwarnings('ignore')

%%time
## Loop all items in summoned prefix

allresults = []

# hack to remove some problematic ones
try:
    sumlist.remove("gleaner/summoned/bcodmo")
except:
    print("bcodmo, note present..")
    
for item in sumlist:
    try:
        print(item)
        fns = oss.ls(item)
        o = []
        # o = [read_and_validate(f) for f in fns]  # need to skip error file ".jsonld"
        for f in fns:
                temp = read_and_validate(f)
                o.append(temp)
        results = dask.compute(*o)
        allresults.append(results)
    except:
        print("ERROR")
        print(item)

# fns = oss.ls(checkPrefix)
# o = [read_and_validate(f) for f in fns]
# results = dask.compute(*o)

gleaner/summoned/cchdo
gleaner/summoned/earthchem
gleaner/summoned/getiedadataorg


<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow.lib.Codec size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)


gleaner/summoned/hydroshare


distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow.lib.Codec size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow.lib.Codec size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full 

gleaner/summoned/ieda


...

ERROR
gleaner/summoned/wikilinkedearth
gleaner/summoned/wwwbco-dmoorg


distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

ERROR
gleaner/summoned/wwwbco-dmoorg
gleaner/summoned/wwwhydroshareorg


distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 6min 13s, sys: 17.6 s, total: 6min 31s
Wall time: 22min 44s


### Single Source

In [ ]:
%%time
## single item

checkPrefix = "gleaner/summoned/magic"
fns = oss.ls(checkPrefix)
o = [read_and_validate(f) for f in fns]
singleresults = dask.compute(*o)

In [ ]:
print(singleresults[1])

### Build an array of graphs based on these.  (or a map?)

In [15]:
allgraphs = []

for ar in allresults:
    rnamespaces = {
        "schema":  "https://schema.org/",
        "shacl":   "http://www.w3.org/ns/shacl#" ,
    }

    kg = kglab.KnowledgeGraph(
        name = "Schema.org shacl eval datagraph",
        base_uri = "https://gleaner.io/id/genid/",
        namespaces = rnamespaces,
    )

    empty = True
    for r in ar:
        if r != None:
            kg.load_rdf_text(data=r, format="ttl")
            empty = False
            
    if not empty:        
        allgraphs.append(kg)

In [16]:
print(len(allgraphs))

10


In [17]:
sparql = """
SELECT ?path ?value ?constraint ?severity ?message ?id ?focus
  WHERE {
    ?id rdf:type shacl:ValidationResult .
    ?id shacl:focusNode ?focus .
    ?id shacl:resultMessage ?message .
    ?id shacl:resultSeverity ?severity .
    ?id shacl:sourceConstraintComponent ?constraint .
    OPTIONAL {
        ?id shacl:resultPath ?path .
    }
    OPTIONAL {
        ?id shacl:value ?value .
    }
  }
"""

for k in allgraphs:
    pdf = k.query_as_df(sparql)
    df = pdf.to_pandas()
    df.style.background_gradient(cmap='Blues')

    if len(df) > 0 :
        dfc = df.groupby('severity').count().reset_index().rename(columns={'path': 'Count'})
        ctst = pd.crosstab(df['message'], df['severity'], margins = True, margins_name = 'Subtotals')

        print("Checking {} objects in {}".format("len of original bucket","NAME" ))
        print(ctst)
        print("-----------------------------------------")

# fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
# fig.suptitle('test results')
i = 1
plt.subplots(figsize=(20, 70))

for k in allgraphs:
    # TODO put a try in here...  
    pdf = k.query_as_df(sparql)
    df = pdf.to_pandas()
    df.style.background_gradient(cmap='Blues')

    if len(df) > 0 :
        dfc = df.groupby('severity').count().reset_index().rename(columns={'path': 'Count'})
        ctst = pd.crosstab(df['message'], df['severity'], margins = True, margins_name = 'Subtotals')

        # sns.set(rc={'figure.figsize':(11.7,8.27)})
        plt.subplot(len(allgraphs),1,i)
        sns.heatmap(ctst, annot=True, fmt=".0f", cmap = sns.cm.crest)
        # axes[i].set_title(i)
        i+=1

plt.show()

AttributeError: 'DataFrame' object has no attribute 'to_pandas'